# Instagram Comments Cleaning

This notebook cleans the comment column in the Instagram comments Excel file.


In [1]:
import pandas as pd
import html
from bs4 import BeautifulSoup
import re
import os

In [2]:
# Load the Excel file with multiple sheets
file_path = 'sgsmu.studentcare_instagram_comments.xlsx'  # Update this path if needed

try:
    # Read all sheets
    sheets = pd.read_excel(file_path, sheet_name=None)
    print("Loaded Excel file with sheets:", list(sheets.keys()))
except FileNotFoundError:
    print("Excel file not found. Trying to load CSV instead.")
    csv_path = '../terrs-EDA_instagram_reddit_telegram/Notebook Analysis/data sources/Instagram - sgsmu.studentcare - Post to Comments.csv'
    df = pd.read_csv(csv_path)
    sheets = {'Sheet1': df}  # Treat as one sheet
    print("Loaded CSV file as single sheet.")

# Dictionary to hold cleaned dataframes
cleaned_sheets = {}

Loaded Excel file with sheets: ['Post 1', 'Post 2', 'Post 3', 'Post 4', 'Post 5', 'Post 6', 'Post 7', 'Post 8']


In [6]:
def clean_comment(text):
    if pd.isna(text) or text == '':
        return ''
    
    # Remove HTML tags
    soup = BeautifulSoup(str(text), 'html.parser')
    text = soup.get_text()
    
    # Unescape HTML entities
    text = html.unescape(text)
    
    # Remove @mentions
    text = re.sub(r'@\w+', '', text)
    
    # Remove emojis and non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [4]:
# Clean each sheet
for sheet_name, df in sheets.items():
    if df.shape[1] >= 4:  # Ensure at least 4 columns (D is 4th, 0-indexed as 3)
        df.iloc[:, 3] = df.iloc[:, 3].apply(clean_comment)  # Column D (index 3)
        cleaned_sheets[sheet_name] = df
        print(f"Cleaned sheet: {sheet_name}")
    else:
        print(f"Warning: Not enough columns in sheet {sheet_name}")
        cleaned_sheets[sheet_name] = df

Cleaned sheet: Post 1
Cleaned sheet: Post 2
Cleaned sheet: Post 3
Cleaned sheet: Post 4
Cleaned sheet: Post 5
Cleaned sheet: Post 6
Cleaned sheet: Post 7
Cleaned sheet: Post 8


In [7]:
# Reset cleaned_sheets and re-clean with updated function
cleaned_sheets = {}

# Clean each sheet again
for sheet_name, df in sheets.items():
    if df.shape[1] >= 4:  # Ensure at least 4 columns (D is 4th, 0-indexed as 3)
        df.iloc[:, 3] = df.iloc[:, 3].apply(clean_comment)  # Column D (index 3)
        cleaned_sheets[sheet_name] = df
        print(f"Re-cleaned sheet: {sheet_name}")
    else:
        print(f"Warning: Not enough columns in sheet {sheet_name}")
        cleaned_sheets[sheet_name] = df

Re-cleaned sheet: Post 1
Re-cleaned sheet: Post 2
Re-cleaned sheet: Post 3
Re-cleaned sheet: Post 4
Re-cleaned sheet: Post 5
Re-cleaned sheet: Post 6
Re-cleaned sheet: Post 7
Re-cleaned sheet: Post 8


In [8]:
# Save the cleaned data to a new Excel file
output_file = 'sgsmu.studentcare_instagram_comments_cleaned.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, df in cleaned_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Cleaned file saved as: {output_file}")

Cleaned file saved as: sgsmu.studentcare_instagram_comments_cleaned.xlsx
